In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src import layer, losses
from src.model import Model
from src.utils import split_batch, train_step, test_step

In [ ]:
data_df = pd.read_csv('energy_efficiency_data.csv')

data_x_df = data_df.drop(['Heating Load', 'Cooling Load'], axis=1)
data_x_df = pd.get_dummies(data_x_df,
                           columns=['Orientation', 'Glazing Area Distribution'],
                           dtype=float)

data_df = pd.concat(
    [data_x_df, data_df['Heating Load']],
    axis=1,
)

data_df


In [ ]:
dataset = np.array(data_df)
np.random.default_rng().shuffle(dataset)

train_ratio = 0.75
train_size = int(len(dataset) * train_ratio)

train_feature = dataset[:train_size, :-1]
train_mean = np.mean(train_feature, axis=0)
train_std = np.std(train_feature, axis=0)
train_feature = (train_feature - train_mean) / train_std

test_feature = (dataset[train_size:, :-1] - train_mean) / train_std

train_target = dataset[:train_size, -1:]
test_target = dataset[train_size:, -1:]

In [ ]:
INPUT_FEATURES = train_feature.shape[1]
OUTPUT_FEATURES = 1

model = Model([
    layer.Linear(INPUT_FEATURES, 10),
    layer.Sigmoid(),
    layer.Linear(10, 10),
    layer.Sigmoid(),
    layer.Linear(10, OUTPUT_FEATURES),
])

mse = losses.MSE()

In [ ]:
EPOCHES = 5000
BATCH_SIZE = 2**7

train_losses, test_losses = [], []
for epoch in range(EPOCHES):

    train_iter_losses, test_iter_losses = [], []
    for x, y in split_batch(train_feature, train_target, BATCH_SIZE):
        train_loss = train_step(x, y, model, mse, learning_rate=0.1)
        train_iter_losses.append(np.sqrt(train_loss))
    train_losses.append(np.mean(train_iter_losses))

    for x, y in split_batch(test_feature, test_target, BATCH_SIZE):
        test_loss = test_step(x, y, model, mse)
        test_iter_losses.append(np.sqrt(test_loss))
    test_losses.append(np.mean(test_iter_losses))

    if (epoch + 1) % 500 == 0:
        print(
            f'Epoch {epoch}: train loss {train_losses[-1]}, test loss {test_losses[-1]}'
        )


In [ ]:
plt.plot(test_target)
plt.plot(model(test_feature))

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)